In [3]:
!tar -xvzf /20news-bydate-matlab.tgz

20news-bydate/matlab/
20news-bydate/matlab/train.data
20news-bydate/matlab/train.label
20news-bydate/matlab/train.map
20news-bydate/matlab/test.data
20news-bydate/matlab/test.label
20news-bydate/matlab/test.map


In [6]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
import math
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score


train_data_x = pd.read_csv(
    "20news-bydate/matlab/train.data", sep=" ", names=["doc_id", "word_id", "freq"],
)
# print(train_data_x.head())
train_data_y = pd.read_csv("20news-bydate/matlab/train.label", names=["labels"],)
# print(train_data_y.head())

test_data_x = pd.read_csv(
    "20news-bydate/matlab/test.data", sep=" ", names=["doc_id", "word_id", "freq"],
)
# print(test_data_x.head())
test_data_y = pd.read_csv("20news-bydate/matlab/test.label", names=["labels"],)
# print(test_data_y.head())

word_cnt = train_data_x[["word_id", "freq"]].groupby(["word_id"], as_index=False).sum()
vocab = list(word_cnt.loc[word_cnt["freq"] > 1000]["word_id"])

train_data_x_processed = train_data_x.loc[
    train_data_x["word_id"].isin(vocab)
].reset_index(drop=True)
train_data_processed = train_data_x_processed.join(
    train_data_y, how="inner", on="doc_id"
)
test_data_x_processed = test_data_x.loc[test_data_x["word_id"].isin(vocab)].reset_index(
    drop=True
)
test_data_processed = test_data_x_processed.join(test_data_y, how="inner", on="doc_id")


# Q1

def get_multinomial_features(data, vocab):
    X = []
    Y = []
    for index, doc_group in data.groupby(["doc_id"]).agg(list).iterrows():
        x = [0] * len(vocab)
        Y.append(list(doc_group["labels"])[0])
        doc_words = list(doc_group.word_id)
        for word_ind in range(len(vocab)):
            if vocab[word_ind] in doc_words:
                x[word_ind] = doc_group.freq[doc_group.word_id.index(vocab[word_ind])]
        X.append(x)
    return np.array(X), np.array(Y)


X_train, Y_train = get_multinomial_features(train_data_processed, vocab)
X_test, Y_test = get_multinomial_features(test_data_processed, vocab)

clf = MultinomialNB()
clf.fit(X_train, Y_train)
print(clf.predict(X_test))
print(clf.score(X_test, Y_test) * 100)

[ 1 16  1 ... 19  1 20]
39.02146380482602
